# This notebook creates confusion matrices and computes F1 scores for the FedNLP Hawk Dove project

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score

In [47]:
def confusion_matrix_f1_score(data, reference):
    # Convert continuous data to categorical data
    data = pd.cut(data, bins=[-np.inf, 0, np.inf], labels=[0, 1])
    reference = pd.cut(reference, bins=[-np.inf, 0, np.inf], labels=[0, 1])

    # Compute confusion matrix and F1 score
    cm = confusion_matrix(reference, data)
    f1 = f1_score(reference, data)

    return cm, f1

In [48]:
gold_label = pd.read_csv('../results/Manual_Labels.csv')
statement = pd.read_csv('../results/statement_full_score.csv')


In [49]:
result = pd.DataFrame({'year':list(statement['date']),
                      'manual':list(gold_label['manual_label']),
                      'gpt4':list(statement['result'])})

In [50]:
confusion_mat = confusion_matrix_f1_score(data = result['gpt4'], reference = result['manual'])

In [55]:
confusion_mat

(array([[115,  11],
        [ 19,  20]]),
 0.5714285714285714)

In [ ]:
sns.heatmap()